In [1]:
using Pkg
Pkg.add("CuArrays")
Pkg.add("CUDAnative")
Pkg.add("CUDAdrv")
Pkg.add("StaticArrays")
Pkg.add("BenchmarkTools")
Pkg.add("Test")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
  [be33ccc6] + CUDAnative v2.2.1
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
  [c5f51814] + CUDAdrv v3.0.1
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
  [90137ffa] + StaticArrays v0.11.0
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
 Installed BenchmarkTools ─ v0.4.2
  Updating `~/.julia/environments/v1.0/Project.toml`
  [6e4b80f9] + BenchmarkTools v0.4.2
 

In [0]:
using CUDAnative, CUDAdrv, CuArrays, StaticArrays, BenchmarkTools, Test

In [2]:
# Kernel to find maximum element in array

function find_max(d_input)
  idx = (blockIdx().x - 1) * blockDim().x + threadIdx().x
  
  if(idx <= size(d_input,1))
    step = 1
    while step < blockDim().x
      if(idx % (step*2) ==1 && idx + step <= size(d_input,1))
        @inbounds d_input[idx] = max(d_input[idx],d_input[idx + step])
      end
      step *= 2
      sync_threads
    end
    if(threadIdx()==1 && idx!=1)
      @inbounds @atomic d_input[1] = max(d_input[1],d_input[idx])
    end
  end
  
  return
end

find_max (generic function with 1 method)

In [68]:
input1 = rand(1:100,2^6)
d_input1 = CuArray(input1)
input2 = input1
d_input2 = CuArray(input2)
const N = size(input1,1)

64

In [69]:
@cuda threads = N find_max(d_input1)
d_max = Array(d_input1)[1]
d_temp1 = CuArray(zeros(Int,N))
d_temp2 = CuArray(zeros(Int,N))
d_ans = CuArray(zeros(Int,N))

64-element CuArray{Int64,1}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [70]:
# Radix sort kernel

function radix_sort(d_input2,d_max,d_temp1,d_temp2,d_ans)
  idx = (blockIdx().x - 1) * blockDim().x + threadIdx().x 
  
  if(idx <= N)
    step = 1
    while step <= d_max
        
      if(d_input2[idx] & step == step)
        d_temp2[idx] = 1
        d_temp1[idx] = 0
      else
        d_temp1[idx] = 1
        d_temp2[idx] = 0
      end
      
      flag = 0 
      if(d_temp1[idx] == 1)
        flag = 1
      end
      sync_threads()
      
      #### Scan on d_temp1 and d_temp2 ####
      Step = 1
      while Step < N
        temp1 = d_temp1[idx]
        temp2 = d_temp2[idx]
        
        if(idx + Step <= N)
          temp1 += d_temp1[idx + Step]
          temp2 += d_temp2[idx + Step]
          d_temp1[idx + Step] = temp1
          d_temp2[idx + Step] = temp2
        end
                
        sync_threads()
        Step *= 2
      end
      ######################################
      
      if(flag == 1)
        d_ans[d_temp1[idx]] = d_input2[idx]
      else
        d_ans[d_temp2[idx] + d_temp1[N]] = d_input2[idx]
      end
      
      d_input2[idx] = d_ans[idx]      
      step *= 2
      sync_threads()
    end
  end
  return
end

radix_sort (generic function with 1 method)

In [0]:
@cuda threads = N radix_sort(d_input2,d_max,d_temp1,d_temp2,d_ans)

In [72]:
@test isapprox(Array(d_input2),sort(input1))

Test Passed

In [73]:
@btime @cuda threads = N radix_sort(d_input2,d_max,d_temp1,d_temp2,d_ans)

  10.509 μs (65 allocations: 2.27 KiB)
